In [1]:
import torch
import syft as sy
import pandas as pd
from syft.lib.python.list import List

### Make the VM and its client

In [2]:
server = sy.VirtualMachine(name="server")
client = server.get_root_client()

In [3]:
print(type(server))
print(server)
print(client.name)
print(type(client))

<class 'syft.core.node.vm.vm.VirtualMachine'>
VirtualMachine: server: <UID: 35a9c640fb1c4f04844cdd4e93521147>
server Client
<class 'syft.core.node.vm.client.VirtualMachineClient'>


### Check out the remote modules

In [6]:
remote_pd = client.pandas
remote_torch = client.torch
remote_np = client.numpy
remote_vision = client.torchvision

In [7]:
print(type(remote_pd))
remote_pd

<class 'syft.ast.module.Module'>


Module:
	.DataFrame -> <syft.ast.klass.Class object at 0x7f903f413700>
	.Series -> <syft.ast.klass.Class object at 0x7f903f413760>
	.CategoricalDtype -> <syft.ast.klass.Class object at 0x7f903f4137c0>
	.Categorical -> <syft.ast.klass.Class object at 0x7f903f413820>
	.json_normalize -> <syft.ast.callable.Callable object at 0x7f903f4138e0>

In [8]:
remote_np

Module:
	.ndarray -> <syft.ast.klass.Class object at 0x7f903f411520>

In [9]:
remote_vision

Module:
	.__version__ -> <syft.ast.static_attr.StaticAttribute object at 0x7f903f535e20>
	.transforms -> Module:
		.Compose -> <syft.ast.klass.Class object at 0x7f903f535ee0>
		.ToTensor -> <syft.ast.klass.Class object at 0x7f903f535f40>
		.Normalize -> <syft.ast.klass.Class object at 0x7f903f3e8040>
		.CenterCrop -> <syft.ast.klass.Class object at 0x7f903f3f1400>
		.ColorJitter -> <syft.ast.klass.Class object at 0x7f903f3f14c0>
		.FiveCrop -> <syft.ast.klass.Class object at 0x7f903f3f1520>
		.Grayscale -> <syft.ast.klass.Class object at 0x7f903f3f15e0>
		.Pad -> <syft.ast.klass.Class object at 0x7f903f3f1640>
		.RandomAffine -> <syft.ast.klass.Class object at 0x7f903f3f1700>
		.RandomApply -> <syft.ast.klass.Class object at 0x7f903f3f17c0>
		.RandomCrop -> <syft.ast.klass.Class object at 0x7f903f3f1880>
		.RandomGrayscale -> <syft.ast.klass.Class object at 0x7f903f3f1940>
		.RandomHorizontalFlip -> <syft.ast.klass.Class object at 0x7f903f3f1a00>
		.RandomPerspective -> <syft.ast.klass

In [10]:
remote_torch

Module:
	.Tensor -> <syft.ast.klass.Class object at 0x7f903f6c5a60>
	.BFloat16Tensor -> <syft.ast.klass.Class object at 0x7f903f6c5ac0>
	.BoolTensor -> <syft.ast.klass.Class object at 0x7f903f6c5b20>
	.ByteTensor -> <syft.ast.klass.Class object at 0x7f903f6c5b80>
	.CharTensor -> <syft.ast.klass.Class object at 0x7f903f6c5be0>
	.DoubleTensor -> <syft.ast.klass.Class object at 0x7f903f6c5c40>
	.FloatTensor -> <syft.ast.klass.Class object at 0x7f903f6c5ca0>
	.HalfTensor -> <syft.ast.klass.Class object at 0x7f903f6c5d00>
	.IntTensor -> <syft.ast.klass.Class object at 0x7f903f6c5d60>
	.LongTensor -> <syft.ast.klass.Class object at 0x7f903f6c5dc0>
	.ShortTensor -> <syft.ast.klass.Class object at 0x7f903f6c5e20>
	.nn -> Module:
		.Parameter -> <syft.ast.klass.Class object at 0x7f903f6c5f40>
		.Module -> <syft.ast.klass.Class object at 0x7f903f616280>
		.Conv2d -> <syft.ast.klass.Class object at 0x7f903f616760>
		.Dropout2d -> <syft.ast.klass.Class object at 0x7f903f616b80>
		.Linear -> <syft.

### Sending server's data to the client

In [22]:
data = {"A": [1, 2]}
pd.json_normalize(data)

,A
0,"[1, 2]"


In [23]:
sy_data = sy.lib.python.Dict(data)
data_ptr = sy_data.send(client)

In [24]:
json_method = remote_pd.json_normalize
dataFrame_ptr = json_method(data_ptr)
print(dataFrame_ptr.get_copy())

        A
0  [1, 2]


### Sending client's data to the server

In [42]:
server = sy.VirtualMachine(name="server")
client = server.get_root_client()
client_data = torch.randn(size=(1, 100))
client_data_ptr = client_data.send(server)

AttributeError: 'VirtualMachine' object has no attribute 'send_immediate_msg_without_reply'

### Checking out client's device (cuda or CPU)

In [43]:
remote_torch = client.torch
client_cuda_ptr = remote_torch.cuda.is_available()
client_cuda = client_cuda_ptr.get_copy()
print(f'client has CUDA? {client_cuda}')

client_device_ptr = remote_torch.device("cuda" if client_cuda else "cpu")
print(f'client device: {client_device_ptr.type.get_copy()}')

client has CUDA? True
client device: cuda


In [ ]:
client_cuda = False
client_cuda_ptr = remote_torch.cuda.is_available()
client_cuda = bool(client_cuda_ptr.get(
                    request_block=True,
                    reason="To run test and inference locally",
                    timeout_secs=5,  # change to something slower
))
print(f'client has CUDA? {client_cuda}')

client_device_ptr = remote_torch.device("cuda" if client_cuda else "cpu")
print(f'client device: {client_device_ptr.type.get_copy()}')

In [8]:
type(device_pointer).__name__

'devicePointer'

In [9]:
device_pointer.id_at_location

<UID: 003d9950bf4f4abf8c0d69190fc40a18>

In [10]:
t1 = torch.tensor([1, 2])
t2 = torch.tensor([1, 3])

In [11]:
syft_list = List([t1, t2])
ptr = syft_list.send(client)

In [12]:
res = ptr.get_copy()
res

[tensor([1, 2]), tensor([1, 3])]

In [33]:
tensor_type='uint8'
op_name = '__add__'

alice = sy.VirtualMachine(name="alice")
alice_client = alice.get_client()


In [34]:
x = torch.tensor([-1, 0, 1, 2, 3, 4])
xp = x.send(alice_client)
op_method = getattr(xp, op_name, None)
target_op_method = getattr(x, op_name)

print(op_method)
print(target_op_method)

<bound method get_run_class_method.<locals>.run_class_method of <syft.proxy.torch.TensorPointer object at 0x7f903f3d5a00>>
<built-in method __add__ of Tensor object at 0x7f903f3a5840>


In [8]:
result = op_method(xp)  # op(xp, xp)
target_result = target_op_method(x)

local_result = result.get()

assert (local_result == target_result).all()

In [5]:
local_result

tensor([-2,  0,  2,  4,  6,  8])